### Scheduling Optimization

In [6]:
#Import relevant data (Qualtrics formatted export)
availability_df = pd.read_csv('Volunteer Availability.csv', skiprows=[0,1])

#Clean column names and format
new_header = ['Name', 'Email', 'Week 1', 'Week 2', 'Week 3', 'Week 4', 'Week 5', 'Week 6', 'Week 7', 'Confirmed by volunteer']
availability_df.columns = new_header
availability_df.insert(0, 'Volunteer ID', availability_df.index + 1)
availability_map = {
    'I am available and prefer this week': 2, 
    'I am available': 1,
    'I am not available': 0
    }

for col in availability_df.columns[3:]:
    availability_df[col] = availability_df[col].map(availability_map)

#Sample weekly volunteer requirements N_j (ITERATION 2: MAKE DYNAMIC)
weekly_volunteers = [3,2,4,5,4,3,2]

availability_df

,Volunteer ID,Name,Email,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Confirmed by volunteer
0,1,Taylor Liu,twliew@uwaterloo.ca,2,0,1,0,2,2,2,NaN
1,2,Taylor Liew,twliew@uwaterloo.ca,1,0,1,1,2,2,2,NaN
2,3,Nicole Thapa,nicole@uwaterloo.ca,1,0,0,0,1,1,0,NaN
3,4,Kate Percy-Robb,kate@uwaterloo.ca,1,1,1,0,0,1,0,NaN
4,5,Harpreet Ghotra,harp@uwaterloo.ca,0,1,0,1,0,0,2,NaN
5,6,Dharsaa Bhagudeva,dhar@uwaterloo.ca,0,0,0,2,2,2,2,NaN
6,7,Bob Smith,twliew@uwaterloo.ca,2,0,1,0,2,2,2,NaN
7,8,Al John,twliew@uwaterloo.ca,1,0,1,1,2,2,2,NaN
8,9,Patrick Star,nicole@uwaterloo.ca,1,0,0,0,1,1,0,NaN
9,10,Sponge Bob,kate@uwaterloo.ca,1,1,1,0,0,1,0,NaN


In [7]:
import pulp

# Create model
model = pulp.LpProblem("Scheduling_Optimization", pulp.LpMaximize)

# Set volunteers and weeks
volunteers = availability_df['Volunteer ID'].tolist()
weeks = list(range(1, 8))

# Decision variables (binary assignment)
week_match = pulp.LpVariable.dicts(
    "x",
    [(i, j) for i in volunteers for j in weeks],
    cat='Binary'
)

# Count available weeks per volunteer
availability_counts = {
    i: (
        availability_df.loc[
            availability_df['Volunteer ID'] == i,
            availability_df.columns[3:]
        ]
        .gt(0)
        .sum(axis=1)
        .values[0]
    )
    for i in volunteers
}

max_avail = max(availability_counts.values())
min_avail = min(availability_counts.values())

# Normalizing the weight so it's a small decimal
weights = {
    i: (max_avail - availability_counts[i]) / (max_avail - min_avail + 1)
    for i in volunteers
}

# -------------------
# Constraints
# -------------------

# Each volunteer must be assigned at least once (if available)
for i in volunteers:
    model += (
        pulp.lpSum(
            week_match[(i, j)] * availability_df.iloc[i-1, j+2]
            for j in weeks
        ) >= 1,
        f"Volunteer_{i}_Min_Placement"
    )

# Each week must have required number of volunteers
for j in weeks:
    model += (
        pulp.lpSum(
            week_match[(i, j)] * availability_df.iloc[i-1, j+2]
            for i in volunteers
        ) == weekly_volunteers[j-1],
        f"Week_{j}_Min_Volunteers"
    )

# -------------------
# Objective Function
# -------------------

model += pulp.lpSum(
    weights[i] * week_match[(i, j)] * availability_df.iloc[i-1, j+2]
    for i in volunteers
    for j in weeks
)

# Solve model
model.solve()

# -------------------
# Results
# -------------------
print("Status:", pulp.LpStatus[model.status])
print("Objective value:", pulp.value(model.objective))

print("\nAssigned Volunteers per Week:")
for i in volunteers:
    for j in weeks:
        if pulp.value(week_match[(i, j)] * availability_df.iloc[i-1, j+2]) > 0.5:
            print(f" - Volunteer {i} assigned to Week {j}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/taylorliew/anaconda3/envs/MSCI446/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/0q/wgx2cs755472fdwvfvs4d2c00000gn/T/756f4d359def4fa5bb262f1ab4747ef2-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/0q/wgx2cs755472fdwvfvs4d2c00000gn/T/756f4d359def4fa5bb262f1ab4747ef2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 24 COLUMNS
At line 263 RHS
At line 283 BOUNDS
At line 334 ENDATA
Problem MODEL has 19 rows, 50 columns and 100 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 13 - 0.00 seconds
Cgl0004I processed model has 19 rows, 50 columns (50 integer (50 of which binary)) and 100 elements
Cutoff increment increased from 1e-05 to 0.2499
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.5
Cbc0038I Pass   1: suminf.

### Export Results

In [8]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill

#Create workbook and sheet
wb = Workbook()
ws = wb.active
ws.title = "Schedule"

#Cols for each week
cell_mapping = [('A', 'B'), ('C', 'D'), ('E', 'F'), ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'N')]

#Style
header_font = Font(bold=True)
header_fill = PatternFill(start_color="daa2fc", end_color="daa2fc", fill_type="solid")

#Fill weekly lists with names and emails
week_number = 1
for start_cell, end_cell in cell_mapping:
    ws.merge_cells(start_cell + str(1) + ':' + end_cell + str(1))
    cell = ws[start_cell + str(1)]
    cell.value = f"Week {week_number}"
    cell.font = header_font
    cell.fill = header_fill
    row = 2

    for i in volunteers:
        if pulp.value(week_match[i, week_number] * availability_df.iloc[i-1, week_number+2]) > 0.5:
            ws[f"{start_cell}{row}"] = availability_df.loc[availability_df['Volunteer ID'] == i, 'Name'].values[0]
            ws[f"{end_cell}{row}"] = availability_df.loc[availability_df['Volunteer ID'] == i, 'Email'].values[0]

            row += 1
    week_number += 1

#Save file
wb.save("Volunteer_Schedule.xlsx")
